# 데이터셋 불러오기

In [1]:
from tqdm.notebook import tqdm
import numpy as np
import pickle

with open('./dataset.pkl', 'rb') as file: # 데이터셋 파일 불러오기
    dataset = pickle.load(file)

# 데이터셋을 훈련셋과 시험셋으로 나눔
trainset = dataset.iloc[:8000].reset_index(drop=True)
testset = dataset.iloc[8000:].reset_index(drop=True)

print('훈련셋')
display(trainset)

print('시험셋')
display(testset)

훈련셋


,data,labels
0,"[101, 115, 109, 78, 163, 159, 6, 143, 62, 82]",0
1,"[98, 161, 29, 155, 12, 76, 108, 89, 126, 87]",1
2,"[100, 55, 158, 55, 185, 176, 89, 147, 20, 1]",0
3,"[75, 16, 190, 67, 88, 180, 97, 123, 87, 95]",0
4,"[10, 84, 170, 66, 113, 146, 66, 169, 77, 46]",0
...,...,...
7995,"[28, 64, 147, 63, 113, 121, 27, 167, 22, 112]",0
7996,"[62, 38, 147, 16, 119, 174, 3, 119, 81, 16]",0
7997,"[96, 52, 155, 85, 191, 158, 33, 122, 25, 99]",0
7998,"[42, 72, 196, 31, 175, 107, 68, 165, 69, 70]",0


시험셋


,data,labels
0,"[187, 174, 26, 190, 9, 77, 150, 77, 188, 111]",1
1,"[81, 57, 95, 27, 148, 140, 47, 182, 93, 31]",0
2,"[172, 114, 47, 105, 85, 54, 136, 58, 197, 87]",1
3,"[76, 38, 167, 51, 176, 118, 10, 150, 56, 78]",0
4,"[115, 34, 192, 30, 100, 87, 99, 90, 113, 47]",0
...,...,...
1995,"[47, 63, 133, 21, 185, 89, 110, 111, 13, 27]",0
1996,"[69, 98, 136, 49, 110, 190, 49, 144, 64, 83]",0
1997,"[154, 124, 15, 100, 52, 11, 199, 19, 99, 92]",1
1998,"[178, 115, 36, 150, 58, 80, 128, 62, 186, 142]",1


# 시험셋 2000개에 대하여 예측을 수행 (Conventional KNN)

In [2]:
predicts = [0 for _ in range(len(testset))]
for idx in tqdm(range(len(testset))):
    input = testset['data'].iloc[idx]
    distances = trainset['data'].apply(lambda x: np.linalg.norm(input-x))
    distances.sort_values(ascending=True, inplace=True)
    indices = distances[:10].index

    neighbors = trainset.iloc[indices]['labels']
    count = neighbors.value_counts()
    key = count.keys()[0]

    predict = key if count.loc[key] >= 5 else ~key
    predicts[idx] = predict

predicts = np.array(predicts)

  0%|          | 0/2000 [00:00<?, ?it/s]

# Classification 성능 측정

In [18]:
GT = testset['labels'].to_numpy()
TP = ((predicts == 1) & (GT == 1)).sum()
FP = ((predicts == 1) & (GT == 0)).sum()
TN = ((predicts == 0) & (GT == 0)).sum()
FN = ((predicts == 0) & (GT == 1)).sum()

accuracy = (TP+TN) / (TP+FP+TN+FN)
precision = TP / (TP+FP)
recall = TP / (TP+FN)
f1 = 2 * precision * recall / (precision + recall)

print(f'{"Accuracy":10s}: {accuracy*100:3.2f}%')
print(f'{"Precision":10s}: {precision*100:3.2f}%')
print(f'{"Recall":10s}: {recall*100:3.2f}%')
print(f'{"F1 score":10s}: {f1:3.2f}')

Accuracy  : 100.00%
Precision : 100.00%
Recall    : 100.00%
F1 score  : 1.00
